##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [2]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道的情感**。

      **评分标准**：
      - 强烈负面 (-5): 极端消极
      - 负面 (-3 ~ -4): 较为消极
      - 轻微负面 (-1 ~ -2): 略微消极
      - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
      - 轻微正面 (+1 ~ +2): 略微积极
      - 正面 (+3 ~ +4): 较为积极
      - 强烈正面 (+5): 极端积极

      **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。所有提供的句子都必须进行情感分析，不得遗漏任何句子**。
      **新闻文章内容:**
      {sentences}

      **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [4]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

C:\Users\82104\AppData\Local\Temp\ipykernel_15832\1462471065.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)


## 데이터 로드

In [9]:
import json

# JSON 파일 로드
file_path = "../data/0310/10/split_by_year/2016.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")

총 445개의 기사 로드 완료


## 감성분석 수행 함수

In [6]:
import time
import random
import json

# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    try:
        # API 요청 전에 딜레이 추가 (랜덤 딜레이)
        delay_time = random.uniform(10, 15)  # 5초에서 10초 사이의 랜덤 딜레이
        time.sleep(delay_time)

        # 기사 본문만 전달
        input_data = {"sentences": article["sentences"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

# 백오프 전략을 위한 재시도 함수
def retry_request(article, max_retries=5, delay_factor=2):
    """백오프 전략을 적용하여 감성 분석 요청을 재시도하는 함수"""
    for attempt in range(max_retries):
        result = analyze_sentiment(article)  # 감성 분석 호출
        if "Error" not in result["sentiment_responses"]:  # 에러가 없으면 바로 반환
            return result
        else:
            print(f"Retry {attempt + 1}/{max_retries} for {article['title']} due to error.")
            time.sleep(random.uniform(30, 40) * delay_factor)  # 재시도 전 대기 (시간 늘리기)
            delay_factor *= 2  # 다음 재시도 때는 더 긴 시간 대기
    return result  # 재시도 후에도 실패하면 마지막 결과 반환

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 순차적으로 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 순차 실행
    for article in data:
        processed_article = retry_request(article)  # 감성 분석 수행
        processed_article = process_article(processed_article)  # 후처리

        if processed_article is not None:
            results.append(processed_article)  # 결과 저장

        # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f" '{article['title']}' 분석 완료.")

    print(f" 전체 기사 감성 분석 완료. {output_file}")


In [11]:
output_path = "../data/0310/results/2016.json"
run_sentiment_analysis(data, output_path)

2025-03-10 16:38:41,372 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['环球网综合报道北京时间上午11时30分朝鲜中央电视台进行特别重大报道宣布6日上午成功进行氢弹核试验。', '全球各国以及国际组织纷纷予以关注和回应。', '中国就朝鲜进行核试验发声明坚决反对 中国外交部发言人华春莹今日在回答记者提问前就此事宣读外交部声明 今天朝鲜民主主义人民共和国不顾国际社会普遍反对再次进行核试验中国政府对此表示坚决反对。', '实现半岛无核化防止核扩散维护东北亚和平稳定。', '是中方的坚定立场。', '我们强烈敦促朝方信守无核化承诺停止采取任何恶化局势的行动。', '维护半岛即东北亚和平稳定符合各方共同利益。', '中方将坚定推进半岛无核化目标坚持通过六方会谈框架解决半岛核问题。', '另有记者问朝鲜方面与中方通报了没有？', '有没有考虑制裁朝鲜？'], 'text': '0\n0\n-3\n0\n0\n-2\n0\n0\n0\n0'}
 '朝鲜宣布氢弹试验成功 中俄美日韩等国纷纷回应-1' 분석 완료.


2025-03-10 16:38:56,696 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['华春莹回答这次朝鲜进行核试验中方事先一无所知。', '关于安理会制裁问题中方将会继续履行国际义务同国际社会一道继续为实现半岛无核化问题做出我们的努力。', '美国白宫尚无法确认核试验真实性 将适当回应朝方挑衅 法新社1月6日报道称白宫方面表示美方目前尚无法确认朝鲜究竟是否进行了核试验。', '白宫方面宣称将对朝鲜方面的挑衅行为给予适当的回应。', '美军方将保持高度警戒 同韩方合作维护半岛安全 美联社报道称美国太平洋司令部发言人表示驻韩美军已经注意到了有关朝鲜核试验的报道美方将对此事保持高度警戒并全力与韩国方面进行合作维护朝鲜半岛的地区安全。', '韩国政府发表声明强烈谴责朝鲜进行核试验 据韩联社1月6日报道韩政府发表声明强烈谴责朝鲜进行核试验。', '安倍朝鲜氢弹试验系重大威胁 绝对不能容忍 日本首相安倍晋三在首相官邸称朝鲜的核试验是对日本安全的重大威胁绝对不能容忍他对此进行强烈谴责。', '据日本时事通信社6日报道安倍6日上午对媒体发言表示朝鲜此次实施氢弹试验对日本的安全是重大的威胁坚决不能够容忍。', '日方将提出强烈谴责。', '朝方的行为明显违反了联合安理会相关决议也是对国际社会核不扩散框架的重大挑战。'], 'text': '0\n0\n0\n0\n0\n-2\n-3\n-3\n-3\n-3'}
 '朝鲜宣布氢弹试验成功 中俄美日韩等国纷纷回应-2' 분석 완료.


2025-03-10 16:39:19,499 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Response format: {'sentences': ['此外由于韩国同意美国在韩部署萨德反导系统导致韩国与中国俄罗斯等邻国矛盾加深所以韩国选择加强与日本之前的合作某种程度上也能减轻周边国家对其造成的压力。', '韩国三大在野党均反对政府签署这一协定。', '他们指出亲信干政事件正引发政局震荡朴槿惠政府支持率一直在历史低位徘徊此时不顾国内反对匆忙签署一份敏感的双边协定做法不妥。', '多数韩国民众也反对签署这一协定。', '盖洛普韩国公布的最新民调结果显示59的受访韩国民众反对签署这一协定支持的人仅占31。', '不少韩国民众和媒体认为日本没有真正反省侵略历史安倍政府在历史问题上的态度尤其让人担忧韩国政府不应该在这种情形下与日本加强军事情报合作。', '日韩情报共享有何价值 韩国与日本媒体曝光的信息显示这份协定涉及情报提供防止泄密的方法以及共享情报的等级等。', '韩媒披露协定签署后韩国有望借助日本的5颗卫星获取包括朝鲜潜艇基地各种弹道导弹基地以及潜射导弹等相关信息。', '日本则可以得到韩国通过特工等人员搜集的人工情报以及韩国在韩朝军事分界线一带通过监听获取的情报等。', '不过有韩国专家认为韩日军事合作现阶段面临诸多掣肘因素存在很多不确定性。'], 'text': '-2\n-2\n-3\n-2\n-2\n-3\n-1\n+1\n+1\n-1'}
 '韩日情报共享能走多远 - 人民日报-10' 분석 완료.


2025-03-10 18:31:00,818 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['其一韩国政府目前掌政能力有限。', '其二韩国国内对加强对日军事合作反对声音强烈。', '其三美国政府换届也将极大影响韩日间军事合作。', '对此袁冲和国内一些专家则认为双边还是存在进一步合作的空间双方签订协议后不只在对朝情报上互通有无在其他领域也可能交换情况是美国同盟体系多边化的重要一步。', '未来韩日安全合作还要视韩国国内形势及东亚地区安全形势发展而定。', '张军社认为日韩军事情报保护协定主要目的就是要加强有关朝鲜情报的共享这种针对第三国的军事同盟是冷战思维的表现必然引起朝鲜方面的激烈反应可能破坏朝鲜半岛的和平稳定对整个东北亚地区的和平稳定也会带来不利影响。', '中国作为朝鲜半岛的近邻也可能因此面临更多安全威胁。', '美日韩军事同盟悬疑 韩日关系正在向着美国乐于看到的方向发展。', '中央党校国际战略研究所教授朝鲜半岛事务专家张琏瑰分析长期以来美国出于自身战略利益考虑力图在东北亚地区打造美日韩三边军事同盟。', '美国一直给韩国和日本施压让他们改善双边关系。'], 'text': '-2\n-3\n-2\n+1\n0\n-4\n-3\n-1\n-2\n-2'}
 '韩日情报共享能走多远 - 人民日报-11' 분석 완료.


2025-03-10 18:31:15,766 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'sentences': ['去年12月在美国的压力下韩日两国坐下来谈判解决慰安妇问题排除了发展关系的一个主要障碍。', '此前有报道称美方曾在10月韩美22会谈时要求韩方在奥巴马任期内必须在部署萨德和签订韩日军事情报保护协定两个问题上取得进展。', '据张琏瑰分析韩日关系在近期可能还会有新的发展。', '12月将在日本举行的三国首脑会议朴槿惠和安倍晋三或许有重大会谈可能会就很多重大问题达成协议。', '上海外国语大学国际关系与公共事务学院特约研究员马尧在接受媒体采访时表示长期以来美国构建美日韩三边军事合作的主要障碍在韩国而韩日军事合作方面的进展意味着三边军事合作的障碍或将不复存在这是韩日在军事领域合作的分水岭。', '但双边同盟目前缺乏相关基础。', '袁冲则认为韩日军事情报合作对中国安全构成一定挑战。', '特别是如果两国加强反导情报的互换将有损中国安全利益。', '朴槿惠政府做出此举动应该是受到来自美国的压力也意在争取韩国国内保守派的支持。', '然而袁冲认为韩日军情合作还有可能引发多米诺骨牌效应使朝鲜半岛和东北亚局势复杂化有可能引发朝鲜方面针锋相对的反击加剧地区各方的矛盾和摩擦导致东北亚这一冷战遗存地区冷上加冷。'], 'text': '0\n-2\n0\n+1\n+2\n-1\n-2\n-3\n-1\n-3'}
 '韩日情报共享能走多远 - 人民日报-12' 분석 완료.
 전체 기사 감성 분석 완료. ../data/0310/results/2016.json
